<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/chapter_10/pagina_485_3th_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/chapter_10/pagina_485_3th_edition.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

Esta es la traducción al español del texto del capítulo 10 de *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow* de Aurélien Géron:





# **MLPs para Regresión**  



En primer lugar, las redes neuronales multicapa (*MLPs*) pueden utilizarse para tareas de regresión. 

Si deseas predecir un único valor (por ejemplo, el precio de una casa dadas varias de sus características), solo necesitas una neurona de salida: su resultado será el valor predicho. 

Para regresión multivariante (es decir, predecir múltiples valores a la vez), necesitas una neurona de salida por cada dimensión de salida. 

Por ejemplo, para localizar el centro de un objeto en una imagen, necesitas predecir coordenadas en 2D, por lo que requieres dos neuronas de salida. 

Si además quieres dibujar un *bounding box* (rectángulo delimitador) alrededor del objeto, necesitas dos números más: el ancho y el alto del objeto. Así, terminarías con cuatro neuronas de salida.  



Scikit-Learn incluye la clase **`MLPRegressor`**, así que vamos a usarla para construir una MLP con tres capas ocultas de 50 neuronas cada una y entrenarla en el conjunto de datos de viviendas de California. 

Para simplificar, utilizaremos la función **`fetch_california_housing()`** de Scikit-Learn para cargar los datos. 

Este conjunto es más simple que el que usamos en el Capítulo 2, ya que solo contiene características numéricas (no incluye la característica *ocean_proximity*) y no tiene valores faltantes.  



El siguiente código comienza cargando y dividiendo el conjunto de datos, luego crea un *pipeline* para estandarizar las características de entrada antes de pasarlas al **`MLPRegressor`**. 

Esto es muy importante para las redes neuronales, ya que se entrenan mediante *gradiente descendente*, y como vimos en el Capítulo 4, el gradiente descendente no converge bien cuando las características tienen escalas muy diferentes.  



Finalmente, el código entrena el modelo y evalúa su error de validación. 

El modelo usa la función de activación **ReLU** en las capas ocultas y una variante del gradiente descendente llamada **Adam** (ver Capítulo 11) para minimizar el *error cuadrático medio (MSE)*, con un poco de regularización **ℓ₂** (controlable mediante el hiperparámetro **`alpha`**):  


In [ ]:

from sklearn.datasets import fetch_california_housing # obtención del 
from sklearn.metrics import root_mean_squared_error # métrica de error
from sklearn.model_selection import train_test_split # división de datos
from sklearn.neural_network import MLPRegressor # red neuronal
from sklearn.pipeline import make_pipeline # pipeline
from sklearn.preprocessing import StandardScaler # escalado de datos


In [ ]:

housing = fetch_california_housing() # obtención del dataset california_housing 


In [ ]:
type(housing)

In [ ]:
housing.keys() 

In [ ]:
housing.target

In [ ]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
    with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing_geron = load_housing_data()
housing_geron.head()

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)


In [ ]:

%%time 
mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42) # instancia MLPRegressor con 3 capas ocultas de 50 neuronas cada una
pipeline = make_pipeline(StandardScaler(), mlp_reg) # creamos un pipeline con escalado estandar y el modelo mlp_reg
pipeline.fit(X_train, y_train) # entrenar el pipeline con los datos de entrenamiento


In [ ]:

y_pred = pipeline.predict(X_valid)
rmse = root_mean_squared_error(y_valid, y_pred)  # ~0.505
10_000*rmse  



Obtenemos un **RMSE de validación** de aproximadamente **0.505**, comparable al rendimiento de un *Random Forest*. ¡Nada mal para un primer intento!  



Cabe destacar que esta MLP **no usa ninguna función de activación en la capa de salida**, por lo que puede generar cualquier valor. Esto suele ser aceptable, pero si necesitas garantizar que la salida siempre sea positiva, deberías usar **ReLU** o **softplus** (una variante suave de ReLU: *softplus(z) = log(1 + exp(z))*). 

La función softplus es cercana a 0 cuando *z* es negativo y cercana a *z* cuando es positivo.  



Por último, si quieres asegurar que las predicciones estén dentro de un rango específico, puedes usar **sigmoid** (para valores entre 0 y 1) o **tanh** (entre –1 y 1) y escalar los objetivos acordemente. Lamentablemente, **`MLPRegressor` no soporta funciones de activación en la capa de salida**.  



# ⚠ **Advertencia**  
Construir y entrenar una MLP estándar con Scikit-Learn en pocas líneas de código es muy conveniente, pero sus capacidades son limitadas. Por eso, más adelante en este capítulo pasaremos a **Keras**.  



La clase **`MLPRegressor`** usa el *error cuadrático medio (MSE)*, que suele ser adecuado para regresión. 

Sin embargo, si hay muchos *outliers* en los datos, podrías preferir el *error absoluto medio (MAE)* o la **pérdida de Huber**, que combina MSE y MAE: es cuadrática para errores menores que un umbral **δ** (típicamente 1) y lineal para errores mayores.

 La parte lineal la hace menos sensible a *outliers* que el MSE, mientras que la parte cuadrática permite una convergencia más rápida que el MAE. No obstante, **`MLPRegressor` solo soporta MSE**.  



La **Tabla 10-1** resume la arquitectura típica de una MLP para regresión.  

| **Hiperparámetro**         | **Valor típico**                                                                 |
|----------------------------|----------------------------------------------------------------------------------|
| N° de capas ocultas        | Depende del problema (usualmente 1 a 5)                                          |
| N° de neuronas por capa    | Depende del problema (usualmente 10 a 100)                                       |
| N° de neuronas de salida   | 1 por dimensión de predicción                                                    |
| Activación en capas ocultas| ReLU                                                                             |
| Activación en salida       | Ninguna, ReLU/softplus (salidas positivas) o sigmoid/tanh (salidas acotadas)     |
| Función de pérdida         | MSE, o Huber si hay *outliers*                                                   |





Espero que esta traducción te sea útil. Si necesitas ajustes o más detalles, ¡avísame!

Aquí tienes una **secuencia didáctica** para practicar la implementación de **MLPs para regresión** en Python, basada en el texto anterior. 

La secuencia está diseñada para avanzar desde conceptos básicos hasta técnicas más avanzadas, con ejercicios prácticos en cada etapa.

---



## **Secuencia Didáctica: MLPs para Regresión en Python**  
**Objetivo:** Implementar y optimizar una red neuronal multicapa (*MLP*) para problemas de regresión usando `scikit-learn` y `Keras`.  



### **1. Introducción a los Datos y Preprocesamiento**  
**Objetivo:** Familiarizarse con el dataset y preparar los datos para el modelo.  

**Ejercicios:**  
1. **Cargar y explorar el dataset**  


In [ ]:

from sklearn.datasets import fetch_california_housing
import pandas as pd

housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df["Target"] = housing.target
df.head()


In [ ]:
df.describe()




2. **División del dataset (train/validation/test)**  


In [ ]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)




3. **Estandarización de características**  


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)




---

### **2. Implementación Básica con `MLPRegressor` (Scikit-Learn)**  
**Objetivo:** Entrenar un primer modelo de MLP y evaluar su rendimiento.  

**Ejercicios:**  
1. **Entrenar un MLP básico**  


In [3]:
from sklearn.neural_network import MLPRegressor

mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], random_state=42)
mlp_reg.fit(X_train_scaled, y_train)



NameError: name 'X_train_scaled' is not defined


2. **Evaluar el modelo (RMSE)**  


In [2]:
from sklearn.metrics import root_mean_squared_error

y_pred = mlp_reg.predict(X_valid_scaled)
rmse = root_mean_squared_error(y_valid, y_pred)
print(f"RMSE: {rmse:.3f}")  # Debería ser ~0.505



NameError: name 'mlp_reg' is not defined


3. **Experimentar con diferentes arquitecturas**  
   - Probar con más/menos capas ocultas.  
   - Cambiar el número de neuronas (ej. `[100, 50]`).  
   - Observar cómo afecta al rendimiento.  





### **3. Optimización del Modelo**  
**Objetivo:** Mejorar el rendimiento del MLP ajustando hiperparámetros.  

**Ejercicios:**  
1. **Ajustar la tasa de aprendizaje y el optimizador**  


In [ ]:
mlp_reg = MLPRegressor(
    hidden_layer_sizes=[50, 50, 50],
    activation="relu",
    solver="adam",  # Usar Adam en lugar de SGD
    learning_rate_init=0.001,  # Tasa de aprendizaje más baja
    max_iter=500,  # Más épocas
    random_state=42
)
mlp_reg.fit(X_train_scaled, y_train)




2. **Regularización (L2) para evitar overfitting** 

In [1]:
 
mlp_reg = MLPRegressor(
    hidden_layer_sizes=[50, 50, 50],
    alpha=0.01,  # Factor de regularización L2
    random_state=42
)
mlp_reg.fit(X_train_scaled, y_train)



NameError: name 'MLPRegressor' is not defined


3. **Early Stopping (usando `validation_fraction`)**  


In [ ]:
mlp_reg = MLPRegressor(
    hidden_layer_sizes=[50, 50, 50],
    early_stopping=True,  # Detener si no mejora
    validation_fraction=0.2,  # 20% de validación
    random_state=42
   )
mlp_reg.fit(X_train_scaled, y_train)






### **4. Implementación con Keras (TensorFlow) para Mayor Flexibilidad**  
**Objetivo:** Usar Keras para modelos más personalizables (ej. activaciones en la capa de salida).  

**Ejercicios:**  
1. **Crear un MLP con Keras**  


In [ ]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(50, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)  # Sin activación (regresión)
])

model.compile(loss="mse", optimizer="adam", metrics=["mse"])
   



2. **Entrenar y evaluar el modelo** 

In [ ]:
 

history = model.fit(
       X_train_scaled, y_train,
       epochs=50,
       validation_data=(X_valid_scaled, y_valid)
   )

y_pred = model.predict(X_test_scaled)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE (Keras): {rmse:.3f}")




3. **Probar diferentes funciones de activación en la salida**  
   - **ReLU/Softplus** (si la salida debe ser positiva): 

In [ ]:
 

keras.layers.Dense(1, activation="softplus")


   - **Sigmoid/Tanh** (si la salida debe estar acotada):

In [ ]:
  
keras.layers.Dense(1, activation="sigmoid")  # Escalar 'y' entre 0 y 1






### **5. Experimentación Avanzada (Opcional)**  
**Objetivo:** Profundizar en técnicas avanzadas para mejorar el modelo.  

**Ejercicios:**  
1. **Usar Batch Normalization** 

In [ ]:
 
model.add(keras.layers.BatchNormalization())



2. **Probar Dropout para regularización** 

In [ ]:
 
model.add(keras.layers.Dropout(0.3))



3. **Optimización con GridSearchCV (Scikit-Learn)**  


In [ ]:

from sklearn.model_selection import GridSearchCV

param_grid = {
    "hidden_layer_sizes": [(50,), (100, 50)],
    "alpha": [0.0001, 0.001, 0.01],
}
grid_search = GridSearchCV(MLPRegressor(), param_grid, cv=3)
grid_search.fit(X_train_scaled, y_train)






## **Resumen de la Secuencia**  
1. **Preprocesamiento**: Estandarización y división de datos.  
2. **Primer modelo con Scikit-Learn**: Entrenamiento y evaluación básica.  
3. **Optimización**: Ajuste de hiperparámetros y regularización.  
4. **Keras**: Modelos más flexibles y personalizables.  
5. **Avanzado**: BatchNorm, Dropout y búsqueda de hiperparámetros.  

Esta secuencia permite pasar de **conceptos básicos** a **técnicas avanzadas**, con ejercicios prácticos en cada paso. ¿Quieres que profundicemos en algún tema en particular?

# Referentes en Deep Learning 

Aquí tienes **10 referentes clave** en *deep learning* para enriquecer tu docencia e investigación en matemáticas, ciencia de datos y métodos cuanti/cuali en la Universidad de Antioquia (Seccional Bajo Cauca). Estos recursos incluyen cursos, libros, investigadores y enfoques innovadores, basados en las tendencias actuales y futuras del campo :





### **1. Cursos y Programas Académicos**  
- **Curso "Bases Matemáticas del Deep Learning" (TECH Universidad Tecnológica)**:  
  Enfocado en fundamentos matemáticos (álgebra lineal, optimización, backpropagation) y aplicaciones prácticas con metodología *Relearning*. Ideal para docencia en matemáticas aplicadas .  
- **Track en Ciencia de Datos (Colegio Bourbaki)**:  
  Cubre desde probabilidad bayesiana hasta redes neuronales avanzadas (CNNs, Transformers, GANs). Incluye proyectos prácticos y evaluación basada en problemas reales .  



### **2. Investigadores y Expertos**  
- **Geoffrey Hinton & Yann LeCun**:  
  Pioneros en redes neuronales y aprendizaje no supervisado. Explora sus trabajos recientes en *capsule networks* y autoaprendizaje (*self-supervised learning*) .  
- **Gary Marcus**:  
  Crítico del *deep learning* puro; promueve modelos híbridos (neuro-simbólicos) que integran razonamiento abstracto y conocimiento previo. Útil para discutir limitaciones del enfoque actual .  



### **3. Frameworks y Herramientas**  
- **TensorFlow/Keras y PyTorch**:  
  Los más usados en docencia. PyTorch es ideal para investigación por su flexibilidad en redes personalizadas .  
- **Libro: "Deep Learning" (Ian Goodfellow et al.)**:  
  Biblia técnica que cubre fundamentos matemáticos (gradientes, regularización) y arquitecturas avanzadas (GANs, RNNs) .  



### **4. Tendencias Futuras**  
- **Aprendizaje Autosupervisado (*Self-Supervised Learning*)**:  
  Reducción de dependencia de datos etiquetados. Ejemplo: modelos de lenguaje como BERT/GPT .  
- **Redes Generativas (GANs y Difusión Estable)**:  
  Aplicaciones en generación de imágenes y datos sintéticos para investigación cualitativa .  



### **5. Métodos Híbridos para Investigación**  
- **Física + Deep Learning**:  
  Modelos que integran leyes físicas en redes neuronales (ej.: predicción climática). Relevante para proyectos interdisciplinarios .  
- **Ética en IA**:  
  Módulos sobre sesgos algorítmicos y justicia social, clave en investigación cualitativa .  



### **6. Recursos Locales (Colombia)**  
- **Grupo GIPI (UdeA)**:  
  Enfoque en ingeniería de procesos con IA. Colaboración potencial para proyectos aplicados .  
- **Sergio Gutiérrez (UdeA)**:  
  Investigador en redes neuronales para detección de intrusiones. Sus publicaciones ofrecen casos prácticos en seguridad .  

---



### **Recomendaciones para Implementación** 

 
1. **En Docencia**: Usa ejemplos de *few-shot learning* (aprendizaje con pocos datos) para clases en entornos con recursos limitados .  


2. **En Investigación**: Combina técnicas cuanti (análisis de RMSE en modelos) con cuali (interpretación de resultados con *attention maps* en CNNs) . 

 
3. **Talleres Prácticos**:  
   - "Cómo entrenar un modelo de NLP con BERT" (usando datasets en español).  
   - "Visualización de gradientes en redes convolucionales" (para explicar matemáticas subyacentes) .  



Estos referentes te permitirán actualizar tus cursos, integrar investigación aplicada y fomentar un enfoque crítico en el uso de IA. 

Para profundizar, explora los enlaces directos a los recursos citados.

# Referentes en la constitución de este cuaderno  

* [End to End machine learning project][https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb]